<a href="https://colab.research.google.com/github/aniishap/ds2002/blob/main/Data_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL Data Processor Project

Target DB Schema: Sakila Database



### Import Necessary Libraries

In [ ]:
!pip install PyMySQL
!pip install mysql.connector
!pip install sqlalchemy
!pip install mysql-connector-python
!pip install pyMysql
!pip install pymongo





In [ ]:
import os
import datetime
import pymongo
import pprint
import pandas as pd
import numpy
import pandas as pd
from sqlalchemy import create_engine


### Declare & Assign Connection Variables to query MySQL using Python


> Add blockquote



In [ ]:


#Set the host name here
#If you are running against a database locally it would be local host or 127.0.0.1
host_name = "database.ds2002.org"
#host_ip = "127.0.0.1"
port = "3306"

user_id = "euv7bw" # my computing id
pwd = "euv7bw!"  #
db_name = "sakila" # db i want to use


### Verify that the connection is succesful

In [ ]:

import pymysql

conn = pymysql.connect(
    host=host_name,
    user=user_id,
    password=pwd
)

print("Connected successfully")
conn.close()

Connected successfully


### Define Functions for Getting Data From and Setting Data Into Databases (MySQL, phpMyAdmin)


In [ ]:

def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()

    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()

    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")

    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')

    connection.close()



### Create & Populate New Dimension Tables: Extracting Data from Sakila Database

SELECT and extract tables chosen for database idea. From the sakila database, I decided to choose these 5 tables: **film, film_category, category, film_actor, and actor**. I decided to choose these 5 ables because they related to each other through primary and foreign keys. The implication of this database is that these tables can keep track and manage various film types and their inventories. Overall, it keeps track of movie rentals and management.

In [ ]:

sql_film = "SELECT * FROM sakila.film;"
df_film = get_dataframe(user_id, pwd, host_name,db_name, sql_film)
df_film.head(5)




,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [ ]:
sql_filmcategory = "SELECT * FROM sakila.film_category;"
df_filmcategory = get_dataframe(user_id, pwd, host_name,db_name, sql_filmcategory)
df_filmcategory.head(5)

,film_id,category_id,last_update
0,1,6,2006-02-15 05:07:09
1,2,11,2006-02-15 05:07:09
2,3,6,2006-02-15 05:07:09
3,4,11,2006-02-15 05:07:09
4,5,8,2006-02-15 05:07:09


In [ ]:
sql_category = "SELECT * FROM sakila.category;"
df_category = get_dataframe(user_id, pwd, host_name,db_name, sql_category)
df_category.head(5)

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [ ]:
sql_filmactor = "SELECT * FROM sakila.film_actor;"
df_filmactor = get_dataframe(user_id, pwd, host_name,db_name, sql_filmactor)
df_filmactor.head(5)

,actor_id,film_id,last_update
0,1,1,2006-02-15 05:05:03
1,1,23,2006-02-15 05:05:03
2,1,25,2006-02-15 05:05:03
3,1,106,2006-02-15 05:05:03
4,1,140,2006-02-15 05:05:03


In [ ]:
sql_actor = "SELECT * FROM sakila.actor;"
df_actor = get_dataframe(user_id, pwd, host_name,db_name, sql_actor)
df_actor.head(5)

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


### Perform Any Necessary Transformations to the DataFrames

Columns were dropped from the 5 dataframes to ensure that data is concise and relevant to the selected database design of tracking movie rental types and management.

In [ ]:
drop_cols = ['last_update', 'original_language_id', 'special_features', 'description']
df_film.drop(drop_cols, axis=1, inplace=True)
df_film.head(2)

,film_id,title,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating
0,1,ACADEMY DINOSAUR,2006,1,6,0.99,86,20.99,PG
1,2,ACE GOLDFINGER,2006,1,3,4.99,48,12.99,G


In [ ]:
drop_cols = ['last_update']
df_filmactor.drop(drop_cols, axis=1, inplace=True)
df_filmactor.head(2)

,actor_id,film_id
0,1,1
1,1,23


In [ ]:
drop_cols = ['last_update']
df_filmcategory.drop(drop_cols, axis=1, inplace=True)
print(df_filmcategory)

     film_id  category_id
0          1            6
1          2           11
2          3            6
3          4           11
4          5            8
..       ...          ...
995      996            6
996      997           12
997      998           11
998      999            3
999     1000            5

[1000 rows x 2 columns]


In [ ]:
drop_cols = ['last_update']
df_actor.drop(drop_cols, axis=1, inplace=True)
df_actor.head(2)

,actor_id,first_name,last_name
0,1,PENELOPE,GUINESS
1,2,NICK,WAHLBERG


In [ ]:
drop_cols = ['last_update']
df_category.drop(drop_cols, axis=1, inplace=True)
df_category.head(2)


,category_id,name
0,1,Action
1,2,Animation


## Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables Using MySQL


In [ ]:
import mysql.connector

# Define connection parameters
host = 'database.ds2002.org'  # Replace with your MySQL host address
user = 'euv7bw'     # Replace with your MySQL username
password = 'euv7bw!' # Replace with your MySQL password
database = 'euv7bw' # Replace with your MySQL database name

# Connect to MySQL
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)



Connected to MySQL database


In [ ]:
cursor = conn.cursor()
cursor.execute("SHOW TABLES")


tables = cursor.fetchall()  # Fetch all tables
for (table_name,) in tables:
    print(table_name)

cursor.close()
conn.close()

actor
category
film
film_actor
film_category
film_profits


### Create film Table

In [ ]:
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)

if conn.is_connected():
    print("Successfully connected to the database")

cursor = conn.cursor()

create_film_query = """
CREATE TABLE film (
  film_id INT AUTO_INCREMENT PRIMARY KEY,
  title VARCHAR(128) NOT NULL,
  release_year YEAR,
  language_id INT,
  rental_duration INT,
  rental_rate DECIMAL(4,2),
  length INT,
  replacement_cost DECIMAL(5,2),
  rating ENUM('G', 'PG', 'PG-13', 'R', 'NC-17'))
"""

try:
    cursor.execute(create_film_query)
    print("Table 'example_table' created successfully.")
except mysql.connector.Error as err:
    print(f"Failed to create table: {err}")

cursor.execute("SHOW TABLES")


tables = cursor.fetchall()
for (table_name,) in tables:
    print(table_name)


cursor.close()
conn.close()


Connected to MySQL database
Successfully connected to the database
Failed to create table: 1050 (42S01): Table 'film' already exists
actor
category
film
film_actor
film_category
film_profits


In [ ]:
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT * FROM film")
        films = cursor.fetchall()
        for film in films:
            print(film)

    except mysql.connector.Error as err:
        print("Error executing SELECT query:", err)
    finally:
        cursor.close()
        conn.close()

Connected to MySQL database
(1, 'ACADEMY DINOSAUR', 2006, 1, 6, Decimal('0.99'), 86, Decimal('20.99'), 'PG')
(2, 'ACE GOLDFINGER', 2006, 1, 3, Decimal('4.99'), 48, Decimal('12.99'), 'G')
(3, 'ADAPTATION HOLES', 2006, 1, 7, Decimal('2.99'), 50, Decimal('18.99'), 'NC-17')
(4, 'AFFAIR PREJUDICE', 2006, 1, 5, Decimal('2.99'), 117, Decimal('26.99'), 'G')
(5, 'AFRICAN EGG', 2006, 1, 6, Decimal('2.99'), 130, Decimal('22.99'), 'G')
(6, 'AGENT TRUMAN', 2006, 1, 3, Decimal('2.99'), 169, Decimal('17.99'), 'PG')
(7, 'AIRPLANE SIERRA', 2006, 1, 6, Decimal('4.99'), 62, Decimal('28.99'), 'PG-13')
(8, 'AIRPORT POLLOCK', 2006, 1, 6, Decimal('4.99'), 54, Decimal('15.99'), 'R')
(9, 'ALABAMA DEVIL', 2006, 1, 3, Decimal('2.99'), 114, Decimal('21.99'), 'PG-13')
(10, 'ALADDIN CALENDAR', 2006, 1, 6, Decimal('4.99'), 63, Decimal('24.99'), 'NC-17')
(11, 'ALAMO VIDEOTAPE', 2006, 1, 6, Decimal('0.99'), 126, Decimal('16.99'), 'G')
(12, 'ALASKA PHANTOM', 2006, 1, 6, Decimal('0.99'), 136, Decimal('22.99'), 'PG')
(13

## Create actor table

In [ ]:
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    cursor = conn.cursor()

    create_actor_table_query = """
    CREATE TABLE actor (
      actor_id INT AUTO_INCREMENT PRIMARY KEY,
      first_name VARCHAR(45) NOT NULL,
      last_name VARCHAR(45) NOT NULL
    )
    """

    try:
        cursor.execute(create_actor_table_query)
        print("Table 'actor' created successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to create table: {err}")
    finally:
        cursor.close()
        conn.close()

Connected to MySQL database
Failed to create table: 1050 (42S01): Table 'actor' already exists


## Create film_actor table

In [ ]:
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    cursor = conn.cursor()

    create_film_actor_table_query = """
    CREATE TABLE film_actor (
      actor_id INT,
      film_id INT,
      FOREIGN KEY (actor_id) REFERENCES actor(actor_id),
      FOREIGN KEY (film_id) REFERENCES film(film_id),
      PRIMARY KEY (actor_id, film_id)
    )
    """

    try:
        cursor.execute(create_film_actor_table_query)
        print("Table 'film_actor' created successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to create table: {err}")
    finally:
        cursor.close()
        conn.close()

Connected to MySQL database
Failed to create table: 1050 (42S01): Table 'film_actor' already exists


## Create category table

In [ ]:
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    cursor = conn.cursor()

    create_category_table_query = """
    CREATE TABLE category (
      category_id INT AUTO_INCREMENT PRIMARY KEY,
      name VARCHAR(45) NOT NULL
    )
    """

    try:
        cursor.execute(create_category_table_query)
        print("Table 'category' created successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to create table: {err}")
    finally:
        cursor.close()
        conn.close()

Connected to MySQL database
Failed to create table: 1050 (42S01): Table 'category' already exists


## Create film_category table

In [ ]:
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    cursor = conn.cursor()

    create_film_category_table_query = """
    CREATE TABLE film_category (
      film_id INT,
      category_id INT,
      FOREIGN KEY (film_id) REFERENCES film(film_id),
      FOREIGN KEY (category_id) REFERENCES category(category_id),
      PRIMARY KEY (film_id, category_id)
    )
    """

    try:
        cursor.execute(create_film_category_table_query)
        print("Table 'film_category' created successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to create table: {err}")
    finally:
        cursor.close()
        conn.close()

Connected to MySQL database
Failed to create table: 1050 (42S01): Table 'film_category' already exists


## Loading the transformed data into new database
### Duplicate errors show because I ran the cell multiple times, but it just means the data is there and will not make duplicates

In [ ]:
import mysql.connector


conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
cursor = conn.cursor()

def insert_dataframe_into_mysql_table(df, table_name):
    cols = ','.join(df.columns)
    placeholders = ','.join(['%s'] * len(df.columns))
    query = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"

    data = [tuple(row) for row in df.values]

    cursor.executemany(query, data)
    conn.commit()

try:
    insert_dataframe_into_mysql_table(df_actor, 'actor')
    print("Data inserted into 'actor' table successfully.")

    insert_dataframe_into_mysql_table(df_category, 'category')
    print("Data inserted into 'category' table successfully.")

    insert_dataframe_into_mysql_table(df_filmcategory, 'film_category')
    print("Data inserted into 'film_category' table successfully.")

    insert_dataframe_into_mysql_table(df_film, 'film')
    print("Data inserted into 'category' table successfully.")

    insert_dataframe_into_mysql_table(df_filmactor, 'film_actor')
    print("Data inserted into 'category' table successfully.")

except mysql.connector.Error as err:
    print(f"Error inserting data: {err}")

# Close cursor and connection
cursor.close()
conn.close()


Error inserting data: 1062 (23000): Duplicate entry '1' for key 'actor.PRIMARY'


In [ ]:
conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
cursor = conn.cursor()

try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")

    cursor = conn.cursor()

    insert_film_query = """
    INSERT INTO film (film_id, title, release_year, language_id, rental_duration, rental_rate, length, replacement_cost, rating)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    for _, row in df_film.iterrows():
        row_data = tuple(row)

        cursor.execute(insert_film_query, row_data)

    conn.commit()

    print("Data inserted into 'film' table successfully.")

except mysql.connector.Error as err:
    print(f"Error inserting data: {err}")
finally:
    cursor.close()
    conn.close()

Connected to MySQL database
Error inserting data: 1062 (23000): Duplicate entry '1' for key 'film.PRIMARY'


In [ ]:
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
    cursor = conn.cursor()

    insert_film_actor_query = """
    INSERT INTO film_actor (actor_id, film_id)
    VALUES (%s, %s)
    """

    for _, row in df_filmactor.iterrows():
        row_data = tuple(row)

        cursor.execute(insert_film_actor_query, row_data)

    conn.commit()

    print("Data inserted into 'film_actor' table successfully.")

except mysql.connector.Error as err:
    print(f"Error inserting data: {err}")
finally:
    cursor.close()
    conn.close()

Connected to MySQL database
Error inserting data: 1062 (23000): Duplicate entry '1-1' for key 'film_actor.PRIMARY'


In [ ]:

try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")

    cursor = conn.cursor()

    insert_film_category_query = """
    INSERT INTO film_category (film_id, category_id)
    VALUES (%s, %s)
    """

    for _, row in df_filmcategory.iterrows():
        row_data = tuple(row)

        cursor.execute(insert_film_category_query, row_data)

    conn.commit()

    print("Data inserted into 'film_category' table successfully.")

except mysql.connector.Error as err:
    print(f"Error inserting data: {err}")
finally:
    cursor.close()
    conn.close()


Connected to MySQL database
Error inserting data: 1062 (23000): Duplicate entry '1-6' for key 'film_category.PRIMARY'


## Extract Data from CSV File

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/aniishap/ds2002/main/Project1/top-500-movies.csv'
original_csv = pd.read_csv(url)
original_csv.head(5)

,rank,release_date,title,url,production_cost,domestic_gross,worldwide_gross,opening_weekend,mpaa,genre,theaters,runtime,year
0,1,2019-04-23,Avengers: Endgame,/movie/Avengers-Endgame-(2019)#tab=summary,400000000,858373000,2797800564,357115007.0,PG-13,Action,4662.0,181.0,2019.0
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,/movie/Pirates-of-the-Caribbean-On-Stranger-Ti...,379000000,241071802,1045713802,90151958.0,PG-13,Adventure,4164.0,136.0,2011.0
2,3,2015-04-22,Avengers: Age of Ultron,/movie/Avengers-Age-of-Ultron#tab=summary,365000000,459005868,1395316979,191271109.0,PG-13,Action,4276.0,141.0,2015.0
3,4,2015-12-16,Star Wars Ep. VII: The Force Awakens,/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=...,306000000,936662225,2064615817,247966675.0,PG-13,Adventure,4134.0,136.0,2015.0
4,5,2018-04-25,Avengers: Infinity War,/movie/Avengers-Infinity-War#tab=summary,300000000,678815482,2048359754,257698183.0,PG-13,Action,4474.0,156.0,2018.0


## Transform CSV files to relate to tables from SAKILA db

In [ ]:
original_csv = original_csv.rename(columns={'genre': 'name'})
original_csv = original_csv.rename(columns={'mpaa': 'rating'})

# rename columns for variable name consistency throughout database

original_csv.head(5)

,rank,release_date,title,url,production_cost,domestic_gross,worldwide_gross,opening_weekend,rating,name,theaters,runtime,year
0,1,2019-04-23,Avengers: Endgame,/movie/Avengers-Endgame-(2019)#tab=summary,400000000,858373000,2797800564,357115007.0,PG-13,Action,4662.0,181.0,2019.0
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,/movie/Pirates-of-the-Caribbean-On-Stranger-Ti...,379000000,241071802,1045713802,90151958.0,PG-13,Adventure,4164.0,136.0,2011.0
2,3,2015-04-22,Avengers: Age of Ultron,/movie/Avengers-Age-of-Ultron#tab=summary,365000000,459005868,1395316979,191271109.0,PG-13,Action,4276.0,141.0,2015.0
3,4,2015-12-16,Star Wars Ep. VII: The Force Awakens,/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=...,306000000,936662225,2064615817,247966675.0,PG-13,Adventure,4134.0,136.0,2015.0
4,5,2018-04-25,Avengers: Infinity War,/movie/Avengers-Infinity-War#tab=summary,300000000,678815482,2048359754,257698183.0,PG-13,Action,4474.0,156.0,2018.0


In [ ]:
import pandas as pd

category_map = {
    "Action": 1,
    "Animation": 2,
    "Children": 3,
    "Classics": 4,
    "Comedy": 5,
    "Documentary": 6,
    "Drama": 7,
    "Family": 8,
    "Foreign": 9,
    "Games": 10,
    "Horror": 11,
    "Music": 12,
    "New": 13,
    "Sci-Fi": 14,
    "Sports": 15,
    "Travel": 16
}

original_csv['category_id'] = original_csv['name'].map(category_map)
original_csv = original_csv.dropna(subset=['category_id'])
original_csv = original_csv.dropna()
original_csv['category_id'] = original_csv['category_id'].astype(int)
original_csv['year'] = original_csv['year'].astype(int)



original_csv.head(5)

,rank,release_date,title,url,production_cost,domestic_gross,worldwide_gross,opening_weekend,rating,name,theaters,runtime,year,category_id
0,1,2019-04-23,Avengers: Endgame,/movie/Avengers-Endgame-(2019)#tab=summary,400000000,858373000,2797800564,357115007.0,PG-13,Action,4662.0,181.0,2019,1
2,3,2015-04-22,Avengers: Age of Ultron,/movie/Avengers-Age-of-Ultron#tab=summary,365000000,459005868,1395316979,191271109.0,PG-13,Action,4276.0,141.0,2015,1
4,5,2018-04-25,Avengers: Infinity War,/movie/Avengers-Infinity-War#tab=summary,300000000,678815482,2048359754,257698183.0,PG-13,Action,4474.0,156.0,2018,1
6,7,2017-11-13,Justice League,/movie/Justice-League-(2017)#tab=summary,300000000,229024295,655945209,93842239.0,PG-13,Action,4051.0,121.0,2017,1
7,8,2015-10-06,Spectre,/movie/Spectre#tab=summary,300000000,200074175,879500760,70403148.0,PG-13,Action,3929.0,148.0,2015,1


In [ ]:
# drop unnecessary columns so that we can create a table focusing on rank, profits and film characteristics

original_csv = original_csv.drop(columns=['release_date', 'url', 'runtime'])
df_filmprofits = original_csv.reset_index(drop=True)  # Drop the old index
change_order = ["rank", "category_id", "title", "production_cost", "domestic_gross",
                  "worldwide_gross", "opening_weekend", "rating", "name", "theaters", "year"]
df_filmprofits = df_filmprofits[change_order]


df_filmprofits.head(5)

,rank,category_id,title,production_cost,domestic_gross,worldwide_gross,opening_weekend,rating,name,theaters,year
0,1,1,Avengers: Endgame,400000000,858373000,2797800564,357115007.0,PG-13,Action,4662.0,2019
1,3,1,Avengers: Age of Ultron,365000000,459005868,1395316979,191271109.0,PG-13,Action,4276.0,2015
2,5,1,Avengers: Infinity War,300000000,678815482,2048359754,257698183.0,PG-13,Action,4474.0,2018
3,7,1,Justice League,300000000,229024295,655945209,93842239.0,PG-13,Action,4051.0,2017
4,8,1,Spectre,300000000,200074175,879500760,70403148.0,PG-13,Action,3929.0,2015


# Create new table in database to load data from transformed CSV file

In [ ]:
import mysql.connector

try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    cursor = conn.cursor()
    create_filmprofits_query = """
    CREATE TABLE film_profits (
        rank_id INT PRIMARY KEY,
        category_id INT,
        title VARCHAR(255) NOT NULL,
        production_cost BIGINT,
        domestic_gross BIGINT,
        worldwide_gross BIGINT,
        opening_weekend INT,
        rating VARCHAR(10),
        name VARCHAR(255) NOT NULL,
        theaters FLOAT,
        year INT,
        FOREIGN KEY (category_id) REFERENCES category(category_id)
    )
    """

    try:
        cursor.execute(create_filmprofits_query)
        print("Table 'film_profits' created successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to create table: {err}")
    finally:
        cursor.close()
        conn.close()


Connected to MySQL database
Table 'film_profits' created successfully.


## Load Transformed CSV Data Into The Created SQL table

In [ ]:
import mysql.connector

try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")

    cursor = conn.cursor()
    insert_filmprofits_query = """
    INSERT INTO film_profits (rank_id, category_id, title, production_cost, domestic_gross, worldwide_gross, opening_weekend, rating, name, theaters, year)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    for _, row in df_filmprofits.iterrows():
        row_data = tuple(row)
        cursor.execute(insert_filmprofits_query, row_data)

    conn.commit()
    print("Data inserted into 'film_profits' table successfully.")

except mysql.connector.Error as err:
    print(f"Error inserting data: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()




Connected to MySQL database
Data inserted into 'film_profits' table successfully.


Extract Data from an API Call and Save to CSV File in Google Colab file directory

In [12]:
import requests
import pandas as pd
import json
import os

def fetch_movie_info(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as err:
        return f"An error occurred: {err}"

    if response_type == 'json':
        result = response.json()
    else:
        result = "An unhandled error has occurred!"

    return result

def write_to_csv(data, file_name):
    df = pd.DataFrame(data['description'])
    df.to_csv(file_name, index=False)
    print(f"Data stored in {file_name}")

valid_url = "https://search.imdbot.workers.dev/?q=avengers" # just an example of extracting by a movie title
response_type = 'json'

movie_data = fetch_movie_info(valid_url, response_type)

if isinstance(movie_data, dict):
    file_name = 'movie_information.csv'
    write_to_csv(movie_data, file_name)
else:
    print(movie_data)

Data stored in movie_information.csv


## Import the Extracted CSV taken from the API Call (saved it and uploaded it to github)

In [16]:
import pandas as pd
url="https://raw.githubusercontent.com/aniishap/ds2002/main/Project1/api_extract.csv"
api_extract=pd.read_csv(url)
api_extract.head(2)

,#TITLE,#YEAR,#IMDB_ID,#RANK,#ACTORS,#AKA,#IMDB_URL,#IMDB_IV,#IMG_POSTER,photo_width,photo_height
0,Avengers: Endgame,2019,tt4154796,547,"Robert Downey Jr., Chris Evans",Avengers: Endgame (2019),https://imdb.com/title/tt4154796,https://t.me/iv?url=https%3A%2F%2Fimdb.com%2Ft...,https://m.media-amazon.com/images/M/MV5BMTc5MD...,1382,2048
1,The Avengers,2012,tt0848228,1261,"Robert Downey Jr., Chris Evans",The Avengers (2012),https://imdb.com/title/tt0848228,https://t.me/iv?url=https%3A%2F%2Fimdb.com%2Ft...,https://m.media-amazon.com/images/M/MV5BNDYxNj...,800,1184


## Transform Data to integrate into database with Sakila data and film_profits data